In [106]:
import ee
import geemap

In [107]:
Map_S5P = geemap.Map()
Map_S5P

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [108]:
point_S5P = ee.Geometry.Rectangle([-122.8003, 37.4831, -122.8036, 37.8288])

image_S5P = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_NO2") \
    .filterBounds(point_S5P) \
    .filterDate('2018-12-01', '2018-12-07') \
    .sort('ALGORITHM_VERSION') \
    .select('stratospheric_NO2_column_number_density')\
    .mean();
    #.select('NO2_column_number_density','tropospheric_NO2_column_number_density','stratospheric_NO2_column_number_density')

vis_params_S5P = {
    'min': 0,
    'max': 0.0002,
    'bands': ['stratospheric_NO2_column_number_density'],
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}
Map_S5P.centerObject(point_S5P, 8)
Map_S5P.addLayer(image_S5P, vis_params_S5P, "Sentinel-5P TROPOMI NO2")


In [109]:
#ee.Date(image_S5P.get('system:time_start')).format('YYYY-MM-dd').getInfo()

In [110]:
#ee.Date(image_S5P.get('system:time_end')).format('YYYY-MM-dd').getInfo()

In [111]:
Map_LANDSAT8 = geemap.Map()
Map_LANDSAT8

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [112]:
point_LANDSAT8 = ee.Geometry.Point([-122.4439, 37.7538])

image_LANDSAT8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point_LANDSAT8) \
    .filterDate('2018-12-01', '2018-12-07') \
    .sort('CLOUD_COVER') \
    .select('B[1-7]') \
    .mean();

vis_params_L8 = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map_S5P.centerObject(point_LANDSAT8, 8)
Map_S5P.addLayer(image_LANDSAT8, vis_params_L8, "Landsat-8")

In [113]:
ground_truth= image_LANDSAT8.addBands(image_S5P)

In [114]:
# Get information about the bands as a list.
bandNames =ground_truth.bandNames()
print('Band names: ', bandNames.getInfo()) # ee.List of band names

Band names:  ['stratospheric_NO2_column_number_density']


In [115]:
# Get information about the bands as a list.
bandNames =image_S5P.bandNames()
print('Band names: ', bandNames.getInfo()) # ee.List of band names

Band names:  ['stratospheric_NO2_column_number_density']


In [116]:
# Get information about the bands as a list.
bandNames =image_LANDSAT8.bandNames()
print('Band names: ', bandNames.getInfo()) # ee.List of band names

Band names:  []


In [117]:
# Get projection information from band 1.
#b1proj = ground_truth.select('B1').projection()
#print('Band 1 projection: \n', b1proj.getInfo()) # ee.Projection object

In [118]:
# Get scale (in meters) information from band 1.
b1scale = ground_truth.select('B1').projection().nominalScale()
print('Band 1 scale: \n', b1scale.getInfo()) # ee.Number

EEException: Image.select: Pattern 'B1' did not match any bands.

In [ ]:
# Get a list of all metadata properties.
properties = ground_truth.propertyNames()
print('Metadata properties:', ) 
properties.getInfo() # ee.List of metadata properties

In [ ]:
#tropimi = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image_S5P.geometry())
Map_LANDSAT8.addLayer(image_S5P, {}, 'TROPOMI')
Map_LANDSAT8

In [ ]:
# Make the training dataset.
points = ground_truth.sample(**{
    'region': ground_truth.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map_S5P.addLayer(points, {}, 'training', False)

In [ ]:
print(points.size().getInfo())

In [ ]:
print(points.first().getInfo())

In [ ]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'stratospheric_NO2_column_number_density'

# Overlay the points on the imagery to get training.
training = ground_truth.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileRandomForest(10).setOutputMode('REGRESSION').train(training, label, bands)

In [ ]:
print(training.first().getInfo())

In [ ]:
vis_params_result = {
    'min': 0,
    'max': 0.0002,
    'bands': ['classification'],
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']}

In [ ]:
# Classify the image with the same bands used for training.
result = ground_truth.select(bands).classify(trained)

# # Display the clusters with random colors.
Map_LANDSAT8.addLayer(result.randomVisualizer(),vis_params_result, 'classfied')
Map_LANDSAT8

In [ ]:
print(result.getInfo())

In [ ]:
vis_params_diff = {
    'min': 0,
    'max': 0.0002,
    'bands': ['stratospheric_NO2_column_number_density'],
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']}

In [ ]:
#// Load a 5-year Landsat 7 composite 2008-2012.
#landsat2008 = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012');

#// Compute multi-band difference between the 2008-2012 composite and the
#// previously loaded 1999-2003 composite.
diff = image_S5P.subtract(result);
#Map.addLayer(diff,
            # {bands: ['', 'B3', 'B2'], min: -32, max: 32}, 'difference');

#// Compute the squared difference in each band.
squaredDifference = diff.pow(2);
#Map.addLayer(squaredDifference,
             #{bands: ['B4', 'B3', 'B2'], max: 1000}, 'squared diff.');

In [ ]:
print(diff.getInfo())

In [ ]:
Map_LANDSAT8.addLayer(diff.randomVisualizer(),vis_params_diff, 'Squared Diff')
Map_LANDSAT8

In [ ]:
print(trained.explain())